In [9]:
import pygame
import random
import sys

# 初始化
pygame.init()
screen_width = 800
screen_height = 600
screen = pygame.display.set_mode((screen_width, screen_height))
pygame.display.set_caption("打砖块游戏")
clock = pygame.time.Clock()

# 颜色定义
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)

# 砖块类
class Brick:
    def __init__(self, x, y):
        self.rect = pygame.Rect(x, y, 80, 30)
        self.color = (random.randint(50, 255), random.randint(50, 255), random.randint(50, 255))
    
    def draw(self, surface):
        pygame.draw.rect(surface, self.color, self.rect)
        pygame.draw.rect(surface, BLACK, self.rect, 2)  # 边框

# 球拍类
class Paddle:
    def __init__(self):
        self.width = 100
        self.height = 15
        self.rect = pygame.Rect(screen_width // 2 - self.width // 2, 
                               screen_height - 40, 
                               self.width, self.height)
        self.speed = 8
        self.color = RED
    
    def draw(self, surface):
        pygame.draw.rect(surface, self.color, self.rect)
    
    def move(self, direction):
        if direction == "left" and self.rect.left > 0:
            self.rect.x -= self.speed
        if direction == "right" and self.rect.right < screen_width:
            self.rect.x += self.speed

# 球类
class Ball:
    def __init__(self):
        self.radius = 10
        self.reset()
        self.color = WHITE
    
    def reset(self):
        self.x = screen_width // 2
        self.y = screen_height // 2
        self.speed_x = 5 * random.choice([-1, 1])
        self.speed_y = -5
    
    def move(self):
        self.x += self.speed_x
        self.y += self.speed_y
        
        # 边界检测
        if self.x <= self.radius or self.x >= screen_width - self.radius:
            self.speed_x *= -1
        if self.y <= self.radius:
            self.speed_y *= -1
    
    def draw(self, surface):
        pygame.draw.circle(surface, self.color, (int(self.x), int(self.y)), self.radius)

# 游戏初始化
def init_game():
    bricks = []
    for row in range(5):
        for col in range(10):
            brick = Brick(col * 82 + 10, row * 35 + 50)
            bricks.append(brick)
    return bricks

# 创建游戏对象
paddle = Paddle()
ball = Ball()
bricks = init_game()
score = 0
font = pygame.font.Font(None, 36)
game_over = False

# 游戏主循环
while True:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()
    
    if not game_over:
        # 球拍控制
        keys = pygame.key.get_pressed()
        if keys[pygame.K_LEFT]:
            paddle.move("left")
        if keys[pygame.K_RIGHT]:
            paddle.move("right")
        
        # 球移动
        ball.move()
        
        # 球与球拍碰撞
        if ball.y + ball.radius >= paddle.rect.top and \
           ball.x >= paddle.rect.left and ball.x <= paddle.rect.right:
            ball.speed_y *= -1
            # 根据击中位置改变反弹角度
            relative_pos = (ball.x - paddle.rect.centerx) / (paddle.width / 2)
            ball.speed_x = relative_pos * 7
        
        # 球与砖块碰撞
        brick_index = None
        for i, brick in enumerate(bricks):
            if (brick.rect.left <= ball.x <= brick.rect.right) and \
               (brick.rect.top <= ball.y <= brick.rect.bottom):
                brick_index = i
                break
        
        if brick_index is not None:
            hit_brick = bricks.pop(brick_index)
            ball.speed_y *= -1
            score += 10
            
            # 如果从侧面击中，改变水平方向
            if ball.x < hit_brick.rect.left or ball.x > hit_brick.rect.right:
                ball.speed_x *= -1
        
        # 检查游戏结束
        if ball.y > screen_height:
            game_over = True
        elif len(bricks) == 0:
            game_over = True
    
    # 绘制
    screen.fill(BLACK)
    
    # 绘制砖块
    for brick in bricks:
        brick.draw(screen)
    
    # 绘制球拍和球
    paddle.draw(screen)
    ball.draw(screen)
    
    # 显示分数
    score_text = font.render(f"分数: {score}", True, WHITE)
    screen.blit(score_text, (10, 10))
    
    # 游戏结束提示
    if game_over:
        if len(bricks) == 0:
            result_text = font.render("恭喜通关! 按R键重新开始", True, WHITE)
        else:
            result_text = font.render("游戏结束! 按R键重新开始", True, WHITE)
        
        screen.blit(result_text, (screen_width//2 - 180, screen_height//2))
        
        keys = pygame.key.get_pressed()
        if keys[pygame.K_r]:
            # 重置游戏
            bricks = init_game()
            ball.reset()
            score = 0
            game_over = False
    
    pygame.display.flip()
    clock.tick(60)